# Current Idea (open to other ideas of course!):

## Micro-Regime Autoencoders (MRAE)

### Problem Statement

Traditional factor models can struggle capturing short-horizon "micro-regimes" in stock price behavior. The Goal of this project is to use a small autoencoder to learn regimes from 20-day *(I just picked a number we can change this obviously)* sliding windows of price/volume features, and then studiy:

- What the regimes look like
- How they transition from one to another
- Whether transitions have predictive power for near-term returns or volatitlity
- Potential comparissons to simple baselines (momentum, volatility)

### Optional Download (You shouldn't need to do this) Please use the link in the readme to get the dataset

In [1]:
# uncomment to download data. 
# Not needed if you have the .parquet file in the data/historical folder

# !python qualifier\utils\download_stock_data.py

### Load Dataset from file

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Set random seeds for reproducibility
np.random.seed(42)

# Data path
PARQUET_PATH = Path("data/historical/all_stocks_historical.parquet")

print(f"Loading data from: {PARQUET_PATH}")
df = pd.read_parquet(PARQUET_PATH)
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn dtypes:")
print(df.dtypes)

Loading data from: data\historical\all_stocks_historical.parquet

DataFrame shape: (89859344, 7)

Column dtypes:
ticker            object
date      datetime64[us]
open             float64
high             float64
low              float64
close            float64
volume           float64
dtype: object


In [ ]:
# standardize and prepare data
print("Available columns:", df.columns.tolist())

# ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

# Sort by ticker and date
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)

print("\n" + "=" * 60)
print("Data Summary:")
print("=" * 60)
print(f"Total rows: {len(df):,}")
print(f"Unique tickers: {df['ticker'].nunique():,}")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Days covered: {(df['date'].max() - df['date'].min()).days:,} days")
print(f"Date column type: {df['date'].dtype}")

Available columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']

Data Summary:
Total rows: 89,859,344
Unique tickers: 7,144
Date range: 1962-01-02 to 2025-11-13
Days covered: 23,326 days
Date column type: datetime64[us]


### Handling Nan values

In [5]:
# original data with nans
print("\n" + "=" * 60)
print("Original Data:")
print("=" * 60)
display(df.head())

# no nan 
print("\n" + "=" * 60)
print("valid (non-null) data:")
print("=" * 60)

# remove rows with nans
cleaned_df = df.dropna(subset=['close', 'volume'])
display(cleaned_df[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']].head())


Original Data:


,ticker,date,open,high,low,close,volume
0,A,1962-01-02,NaN,NaN,NaN,NaN,NaN
1,A,1962-01-03,NaN,NaN,NaN,NaN,NaN
2,A,1962-01-04,NaN,NaN,NaN,NaN,NaN
3,A,1962-01-05,NaN,NaN,NaN,NaN,NaN
4,A,1962-01-08,NaN,NaN,NaN,NaN,NaN



valid (non-null) data:


,ticker,date,open,high,low,close,volume
9539,A,1999-11-18,27.245214,29.939798,23.951837,26.347021,62546380.0
9540,A,1999-11-19,25.710805,25.748231,23.839568,24.176392,15234146.0
9541,A,1999-11-22,24.737756,26.347021,23.989261,26.347021,6577870.0
9542,A,1999-11-23,25.448832,26.122478,23.951841,23.951841,5975611.0
9543,A,1999-11-24,24.026691,25.112009,23.951841,24.588062,4843231.0


### Trimming data to a specific start date

In [6]:
# trimming the df_valid to start at the year 2000
cleaned_trimmed_df = cleaned_df[cleaned_df['date'] >= '2000-01-01'].copy()
print("\n" + "=" * 60)
print("Filtered data (Year 2000 onwards):")
print("=" * 60)
print(f"Total rows: {len(cleaned_trimmed_df):,}")
print(f"Date range: {cleaned_trimmed_df['date'].min().date()} to {cleaned_trimmed_df['date'].max().date()}")
display(cleaned_trimmed_df[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']].head())


Filtered data (Year 2000 onwards):
Total rows: 21,521,135
Date range: 2000-01-03 to 2025-11-13


,ticker,date,open,high,low,close,volume
9569,A,2000-01-03,47.155195,47.267470,40.343889,43.113323,4674353.0
9570,A,2000-01-04,40.792992,41.242092,38.772056,39.819950,4765083.0
9571,A,2000-01-05,39.670253,39.819954,36.114903,37.349918,5758642.0
9572,A,2000-01-06,36.900802,37.125352,34.805015,35.927757,2534434.0
9573,A,2000-01-07,35.366397,39.483121,35.328974,38.921749,2819626.0


### Removing stocks that do not have at least 1000 days of data and Filtering for highest volume stocks

In [7]:
# two-stage universe selection/filtering to shrink for MVP
# 1. quality filter: remove tickers with insufficient history
# 2. liquidity: select top N by average volume from quality-filtered set


min_days_required = 1000  # ~4 years: enough for windows + train/val/test splits
top_n = 50  # target universe size

print("\n" + "=" * 60)
print(f"Stage 1: Quality Filter (min {min_days_required} trading days)")
print("=" * 60)

# count days per ticker in the full trimmed dataset
rows_per_ticker_all = cleaned_trimmed_df.groupby('ticker').size()
valid_tickers = rows_per_ticker_all[rows_per_ticker_all >= min_days_required].index.tolist()

print(f"Tickers before quality filter: {len(rows_per_ticker_all):,}")
print(f"Tickers after quality filter:  {len(valid_tickers):,}")
print(f"Tickers removed: {len(rows_per_ticker_all) - len(valid_tickers):,}")

# filter to quality tickers only
quality_filtered_df = cleaned_trimmed_df[cleaned_trimmed_df['ticker'].isin(valid_tickers)].copy()

print("\n" + "=" * 60)
print(f"Stage 2: Liquidity Ranking (top {top_n} by avg volume)")
print("=" * 60)

# average volume on quality-filtered stocks
avg_volume_by_ticker = quality_filtered_df.groupby('ticker')['volume'].mean().sort_values(ascending=False)

# select top N from the filtered universe
universe = avg_volume_by_ticker.head(top_n).index.tolist()



Stage 1: Quality Filter (min 1000 trading days)
Tickers before quality filter: 7,093
Tickers after quality filter:  5,182
Tickers removed: 1,911

Stage 2: Liquidity Ranking (top 50 by avg volume)


***ONLY USE 'data' IN THE NEXT STEPS. IT IS CLEANED AND TRIMMED***

In [8]:

# create FINAL FINAL dataset
data = quality_filtered_df[quality_filtered_df['ticker'].isin(universe)].copy()
data = data.sort_values(['ticker', 'date']).reset_index(drop=True)

print(f"Selected universe size: {len(universe)} tickers (target: {top_n})")
print(f"{len(universe)} tickers have >= {min_days_required} days of history")

print("-" * 60)
for i, (ticker, vol) in enumerate(avg_volume_by_ticker.head(10).items(), 1):
    print(f"  {i:2d}. {ticker:6s} - {vol:>15,.0f} shares/day")

Selected universe size: 50 tickers (target: 50)
50 tickers have >= 1000 days of history
------------------------------------------------------------
   1. NVDA   -     600,617,217 shares/day
   2. AAPL   -     374,911,010 shares/day
   3. AMZN   -     116,037,570 shares/day
   4. GOOGL  -     112,346,485 shares/day
   5. GOOG   -     110,036,509 shares/day
   6. TSLA   -      96,896,489 shares/day
   7. TQQQ   -      84,554,877 shares/day
   8. BAC    -      77,327,168 shares/day
   9. QQQ    -      66,639,305 shares/day
  10. NIO    -      61,121,488 shares/day


In [9]:
print("\n" + "=" * 60)
print(f"Final Universe: {len(universe)} stocks")
print("=" * 60)
print(f"Total rows: {len(data):,}")
print(f"Unique tickers: {data['ticker'].nunique()}")
print(f"Date range: {data['date'].min().date()} to {data['date'].max().date()}")


print("\n" + "-" * 60)
print("Data Completeness (per ticker):")
print("-" * 60)
rows_per_ticker = data.groupby('ticker').size()
print(f"  Min:    {rows_per_ticker.min():>5,} days  (>= {min_days_required} guaranteed)")
print(f"  Median: {rows_per_ticker.median():>5,.0f} days")
print(f"  Mean:   {rows_per_ticker.mean():>5,.0f} days")
print(f"  Max:    {rows_per_ticker.max():>5,} days")

print("\n" + "=" * 60)
print("Selected Tickers (sorted by avg volume):")
print(f"  {', '.join(universe)}")

display(data.head(10))


Final Universe: 50 stocks
Total rows: 244,098
Unique tickers: 50
Date range: 2000-01-03 to 2025-11-13

------------------------------------------------------------
Data Completeness (per ticker):
------------------------------------------------------------
  Min:    1,007 days  (>= 1000 guaranteed)
  Median: 6,506 days
  Mean:   4,882 days
  Max:    6,507 days

Selected Tickers (sorted by avg volume):
  NVDA, AAPL, AMZN, GOOGL, GOOG, TSLA, TQQQ, BAC, QQQ, NIO, PLTR, INTC, MSFT, F, SOFI, CSCO, OPEN, AMD, PFE, RIVN, T, SNAP, CMG, WMT, META, EBAY, ORCL, AVGO, ITUB, CMCSA, CSX, LRCX, HPQ, UBER, WFC, VALE, MU, BBAI, TLRY, GRAB, NOK, RGTI, HOOD, JPM, BABA, C, AMAT, PBR, XOM, BBD


,ticker,date,open,high,low,close,volume
0,AAPL,2000-01-03,0.786328,0.843498,0.762428,0.839281,5.357968e+08
1,AAPL,2000-01-04,0.811633,0.829440,0.758680,0.768521,5.123776e+08
2,AAPL,2000-01-05,0.777892,0.828971,0.772269,0.779767,7.783216e+08
3,AAPL,2000-01-06,0.795700,0.802260,0.712287,0.712287,7.679728e+08
4,AAPL,2000-01-07,0.723534,0.757274,0.716037,0.746027,4.607344e+08
5,AAPL,2000-01-10,0.764771,0.766646,0.710413,0.732906,5.050640e+08
6,AAPL,2000-01-11,0.719316,0.745090,0.678547,0.695417,4.415488e+08
7,AAPL,2000-01-12,0.712287,0.716036,0.648556,0.653711,9.760688e+08
8,AAPL,2000-01-13,0.708421,0.740403,0.693543,0.725408,1.032685e+09
9,AAPL,2000-01-14,0.749776,0.766646,0.745090,0.753056,3.903760e+08


### Feature engineering (Updated to ensure no lookahead bias or leakage happens because of returns/feature calculations)

In [10]:
feat_df = data.copy()
feat_df = feat_df.sort_values(['ticker', 'date']).reset_index(drop=True)

# 1. log price & 1-day log return 
feat_df['log_price'] = np.log(feat_df['close'])
feat_df['log_ret_1d'] = feat_df.groupby('ticker')['log_price'].diff()

# 2. rolling volatilities of returns 
feat_df['vol_5d'] = (
    feat_df.groupby('ticker')['log_ret_1d']
           .rolling(window=5, min_periods=5)
           .std()
           .reset_index(level=0, drop=True)
)

feat_df['vol_20d'] = (
    feat_df.groupby('ticker')['log_ret_1d']
           .rolling(window=20, min_periods=20)
           .std()
           .reset_index(level=0, drop=True)
)

# 3. vol z-score with expanding mean/std per ticker
g = feat_df.groupby('ticker')['volume']
exp_mean = g.expanding().mean().reset_index(level=0, drop=True)
exp_std  = g.expanding().std(ddof=0).reset_index(level=0, drop=True)

feat_df['volume_mean_exp'] = exp_mean
feat_df['volume_std_exp']  = exp_std
feat_df['volume_z'] = (feat_df['volume'] - feat_df['volume_mean_exp']) / feat_df['volume_std_exp']

# 4. intraday range
feat_df['range_frac'] = (feat_df['high'] - feat_df['low']) / feat_df['close']

# Keep only needed columns
feature_cols = ['log_ret_1d', 'vol_5d', 'vol_20d', 'volume_z', 'range_frac']

feat_df = feat_df[['ticker', 'date'] + feature_cols]

# drop nan rows because of features
feat_df = feat_df.dropna(subset=feature_cols).reset_index(drop=True)

print("Feature DF shape:", feat_df.shape)
feat_df.head()

Feature DF shape: (243098, 7)


,ticker,date,log_ret_1d,vol_5d,vol_20d,volume_z,range_frac
0,AAPL,2000-02-01,-0.034317,0.037641,0.054475,-0.887578,0.049875
1,AAPL,2000-02-02,-0.014443,0.037796,0.050977,-0.435508,0.051866
2,AAPL,2000-02-03,0.044535,0.048172,0.051917,-0.391585,0.038718
3,AAPL,2000-02-04,0.044373,0.035449,0.048214,-0.532542,0.059028
4,AAPL,2000-02-07,0.054615,0.040390,0.048611,-0.470666,0.072876
